In [51]:
import pandas as pd
import numpy as np
import time

import urllib.request
import urllib.error

from bs4 import BeautifulSoup

from tqdm.notebook import trange, tqdm

import random

In [42]:
patronite_id = 86207

In [28]:
def grab_page(patronite_id):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-28-41a29255bd24>, line 2)

In [43]:
page_url = f"https://patronite.pl/profil/{patronite_id}/patron"

# zapytanie razem z UserAgent
req = urllib.request.Request(page_url,
                                headers={
                                    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
                                })

page = urllib.request.urlopen(req)


In [44]:
# wczytanie strony i przerobienie na string
page_doc = page.read().decode('utf-8')

# parser HTMLa z BS
soup = BeautifulSoup(page_doc, 'html.parser')

In [45]:
# nazwa usera
user_name = soup.find_all("h1")[0].getText().strip()

In [46]:
# od kiedy w Partonite
el = soup.find_all("div", attrs={"class": "author__bio--joinDate"})[0].getText().strip()
el = el.replace('W Patronite od ', "")
el = time.strptime(el, "%d.%m.%Y")
user_date = (el.tm_year, el.tm_mon, el.tm_mday)

In [56]:
dct = {
    "id": patronite_id,
    "name": user_name,
    "registration_year": user_date[0],
    "registration_month": user_date[1],
    "registration_day": user_date[2],
    "patrons": np.NaN,
    "dotations_per_month": np.NaN,
    "total_dotations": np.NaN
}

In [57]:
user_numbers = [0, 0, 0]
for i, elem in enumerate(soup.find("div", attrs={"class": "author__stats--wrapper"}).children):
    if len(elem) > 1:
        
        ciag = elem.getText().strip().replace("\n", " ")
        if "łącznie" in ciag:
            liczba = ciag.replace(" zł łącznie", "").replace(" ", "")
            dct['total_dotations'] = float(liczba)

        if "miesięcznie" in ciag:
            liczba = ciag.replace(" zł miesięcznie", "").replace(" ", "")
            dct['dotations_per_month'] = float(liczba)

        if "patronów" in ciag:
            liczba = ciag.replace("patronów", "").replace(" ", "")
            dct['patrons'] = float(liczba)


In [58]:
dct

{'id': 86207,
 'name': 'Michał Sikorski',
 'registration_year': 2018,
 'registration_month': 1,
 'registration_day': 10,
 'patrons': nan,
 'dotations_per_month': nan,
 'total_dotations': 12855.0}

In [59]:
pd.DataFrame(dct, index=[0])

,id,name,registration_year,registration_month,registration_day,patrons,dotations_per_month,total_dotations
0,86207,Michał Sikorski,2018,1,10,NaN,NaN,12855.0


In [73]:
today = time.mktime((time.gmtime().tm_year, time.gmtime().tm_mon, time.gmtime().tm_mday, 0,0,0, 0,0,0))

full_df = pd.DataFrame()
id = 0
added = 0
stop = False
saved = False

while not stop:
    id += 1
    temp_df = grab_page(id) 
    if temp_df is not None:
        full_df = full_df.append(temp_df)
        added += 1
        saved = False
        print(f"Added {added} (after {id} iterations) user: {temp_df.name[0]}")
        if today == time.mktime((temp_df.registration_year[0],
                                 temp_df.registration_month[0],
                                 temp_df.registration_day[0],
                                 0,0,0,
                                 0,0,0)):
            stop = True
    
    if added % 10 == 0 and not saved:
        full_df.to_pickle(f"data/{added}_{id}.pkl")
        print(f'Saved to file "data/{added}_{id}.pkl"')
        saved = True


5 HTTP Error 404: Not Found
Added 1 (after 6 iterations) user: KOLOROFONIA
13 HTTP Error 404: Not Found
20 HTTP Error 404: Not Found
26 HTTP Error 404: Not Found
27 HTTP Error 404: Not Found
Added 2 (after 35 iterations) user: Maciej Łukaszuk
42 HTTP Error 404: Not Found
48 HTTP Error 404: Not Found
66 HTTP Error 404: Not Found
72 HTTP Error 404: Not Found
Added 3 (after 86 iterations) user: Kaja Mikoszewska
87 HTTP Error 404: Not Found
90 HTTP Error 404: Not Found
116 HTTP Error 404: Not Found
136 HTTP Error 404: Not Found
140 HTTP Error 404: Not Found
Added 4 (after 153 iterations) user: Współczesny polski komiks
Added 5 (after 156 iterations) user: Jacek Wesołowski
199 HTTP Error 404: Not Found
Added 6 (after 212 iterations) user: Pracownia Cioci Kot
217 HTTP Error 404: Not Found
232 HTTP Error 404: Not Found
Added 7 (after 242 iterations) user: Astrofaza
Added 8 (after 260 iterations) user: Piotr Kuldanek
264 HTTP Error 404: Not Found
Added 9 (after 327 iterations) user: Anna Kłoso

KeyboardInterrupt: 

In [74]:
full_df = pd.read_pickle("data/20_661.pkl")

In [119]:
id = 2900
added = 34
full_df = pd.read_pickle(f"data/{added}_{id}.pkl").drop_duplicates()
full_df

,id,name,registration_year,registration_month,registration_day,patrons,dotations_per_month,total_dotations
0,6,KOLOROFONIA,2015,9,14,51,1357.0,NaN
0,35,Maciej Łukaszuk,2015,10,9,5,60.0,3590.0
0,86,Kaja Mikoszewska,2015,11,28,116,1885.0,NaN
0,153,Współczesny polski komiks,2015,12,6,0,0.0,770.0
0,156,Jacek Wesołowski,2015,12,6,22,454.0,28867.0
0,212,Pracownia Cioci Kot,2016,6,19,1,100.0,3120.0
0,242,Astrofaza,2016,10,3,293,7705.0,NaN
0,260,Piotr Kuldanek,2016,1,10,22,480.0,27310.0
0,327,Anna Kłosowska,2016,1,26,46,720.0,17780.0
0,328,Razer,2016,10,4,12,310.0,14348.0


In [81]:
id = 1101
added = 30
saved = False

if ((id % 100 == 0) or (added % 10 == 0)) and not saved:
    print("zapisuje")

zapisuje


In [90]:
full_df.drop_duplicates().to_pickle(f"data/{added}_{id}.pkl")

In [118]:
random.choice([0.25, 0.5, 0.75, 1])

1

In [110]:
seq(0, 2, 0.25)

NameError: name 'seq' is not defined